<a href="https://colab.research.google.com/github/TCU-DCDA/WRIT20833-2025/blob/main/notebooks/codeAlongs/WRIT20833_Topic_Modeling_Gensim_F25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling with Gensim LDA
## Discovering Hidden Themes in Cultural Texts

Welcome to topic modeling! Today we'll learn to uncover **hidden themes and topics** in large collections of text using machine learning.

### 🎯 What You'll Learn:
- **Understand topic modeling** as a distant reading technique
- **Install and use Gensim LDA** for discovering hidden topics
- **Preprocess text** appropriately for topic modeling (different from sentiment analysis!)
- **Interpret topic word lists** and assign meaningful labels
- **Think critically** about algorithmic theme detection

### 🔗 Connection to Your Work:
This prepares you for **HW4-2**, where you'll apply topic modeling to your own dataset and integrate findings with your HW4-1 term frequency and sentiment analysis.

## Part 1: From Word Counting to Theme Discovery

### Your Text Analysis Journey So Far

**HW1: Term Frequency**
- Question: "What words appear most often?"
- Output: List of frequent words ("love", "good", "time")
- Insight: Surface-level word patterns

**HW4-1: Sentiment Analysis**
- Question: "What emotions do these texts express?"
- Output: Positive/negative/neutral scores
- Insight: Emotional tone and connotation

**Today: Topic Modeling**
- Question: "What hidden themes and subjects are in this collection?"
- Output: Groups of related words that form coherent topics
- Insight: Deep thematic structure and subject patterns

### 🤔 The Critical Question

**Can an algorithm truly discover cultural "themes"?**

Topic modeling doesn't "understand" culture—it finds statistical patterns in word co-occurrence. When you see topics emerge:
- The algorithm is clustering words that appear together frequently
- **YOU** must interpret whether those clusters represent meaningful cultural themes
- This is where humanistic interpretation meets computational analysis

### 📖 What is Topic Modeling?

**Topic modeling** uses machine learning to discover abstract "topics" in a collection of documents.

**How it works**:
1. Assumes each document contains a mixture of topics
2. Assumes each topic is a collection of related words
3. Uses statistics to reverse-engineer what those topics might be

**Example**: Analyzing 100 movie reviews
- **Topic 1**: plot, story, narrative, character, ending (→ *Storytelling*)
- **Topic 2**: acting, performance, cast, actor, role (→ *Performance*)
- **Topic 3**: visual, effects, cinematography, scene, shot (→ *Visuals*)
- **Topic 4**: boring, slow, waste, terrible, disappointing (→ *Negative Critiques*)

**Your job as researcher**: Interpret word lists and decide if they form coherent themes!

In [ ]:
# Setup: Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import re

print("📚 Basic libraries imported - ready for topic modeling!")

In [ ]:
# Install required libraries for topic modeling
!pip install gensim
!pip install pyLDAvis
!pip install nltk

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

print("✅ Topic modeling libraries installed!")

In [ ]:
# Import Gensim and related libraries
import gensim
from gensim import corpora
from gensim.models import LdaModel
from gensim.models import CoherenceModel
from nltk.stem import WordNetLemmatizer
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

print("✅ Gensim and topic modeling tools ready!")
print(f"Gensim version: {gensim.__version__}")

## Part 2: Topic Modeling with Simple Cultural Examples

Let's start with a small, clear example to understand how topic modeling works:

In [ ]:
# Create sample cultural data - museum reviews with clear themes
museum_reviews = [
    # Art-focused reviews
    "The paintings were incredible. Van Gogh's work was beautifully displayed with excellent lighting.",
    "Amazing art collection! The modern art gallery featured fantastic paintings and sculptures.",
    "Loved the impressionist paintings. The colors and brushwork were stunning.",
    
    # History-focused reviews
    "The ancient artifacts were fascinating. Egyptian mummies and pottery from thousands of years ago.",
    "Great historical exhibits! Medieval weapons, armor, and manuscripts were well preserved.",
    "Incredible history museum. Roman coins, Greek pottery, and ancient tools.",
    
    # Family/kids reviews
    "Perfect for families! The kids loved the interactive exhibits and hands-on activities.",
    "Children had a blast. Interactive dinosaur exhibit kept them engaged for hours.",
    "Great for kids. Educational activities and fun interactive displays throughout.",
    
    # Facility/practical reviews
    "The museum cafe was expensive. Gift shop had limited options. Parking was difficult.",
    "Long lines to enter. Crowded galleries. Overpriced admission tickets and food.",
    "Clean facilities and friendly staff. Good accessibility for wheelchairs."
]

print("🏛️ Sample Museum Reviews Dataset")
print(f"Total reviews: {len(museum_reviews)}")
print("\n🤔 PREDICTION TIME:")
print("What topics do YOU expect topic modeling to discover?")
print("Write down 3-4 topic predictions before we run the analysis...")

### 📝 Your Predictions:

**Topic 1**: _____________________

**Topic 2**: _____________________

**Topic 3**: _____________________

**Topic 4**: _____________________

## Part 3: Text Preprocessing for Topic Modeling

### ⚠️ Different Analysis = Different Preprocessing

**For VADER Sentiment Analysis (HW4-1)**:
- ✅ Keep punctuation ("good!!!" vs "good")
- ✅ Keep capitalization ("AMAZING" vs "amazing")
- ✅ Keep emojis (😍, ❤️)

**For Topic Modeling (Today)**:
- ❌ Remove punctuation (not meaningful for topics)
- ❌ Remove capitalization ("Art" and "art" are same word)
- ✅ Lemmatize words ("running" → "run", "paintings" → "painting")
- ✅ Remove short words ("a", "an", "the")
- ✅ Remove domain-specific stopwords

**Why the difference?** 
- Sentiment = emotional intensity matters ("good" vs "GOOD!!!")
- Topics = semantic meaning matters ("painting", "paintings", "painted" = same concept)

In [ ]:
# Enhanced stopwords list for topic modeling
stopwords = [
    # Basic English stopwords
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours",
    "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers",
    "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves",
    "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is",
    "are", "was", "were", "be", "been", "being", "have", "has", "had", "having",
    "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or",
    "because", "as", "until", "while", "of", "at", "by", "for", "with", "about",
    "against", "between", "into", "through", "during", "before", "after", "above",
    "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under",
    "again", "further", "then", "once", "here", "there", "when", "where", "why",
    "how", "all", "both", "each", "few", "more", "most", "other", "some", "such",
    "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very",
    "s", "t", "can", "will", "just", "don", "should", "now", "ve", "ll", "amp",
    
    # Additional words that don't help with topics
    "also", "would", "could", "get", "go", "one", "two", "see", "time", "way",
    "may", "said", "say", "new", "first", "last", "long", "little", "much",
    "well", "still", "even", "back", "good", "many", "make", "made", "us", "really",
    
    # Museum-specific stopwords (domain-specific)
    "museum", "exhibit", "exhibition", "visit", "visited", "visitor", "review"
]

print(f"✅ Stopwords list loaded: {len(stopwords)} words to filter out")

In [ ]:
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

def preprocess_for_topics(text):
    """
    Aggressive text preprocessing for topic modeling
    """
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation and split into words
    words = re.findall(r'\b[a-z]+\b', text)
    
    # Remove stopwords and short words (< 3 characters)
    words = [word for word in words if word not in stopwords and len(word) >= 3]
    
    # Lemmatize words (reduce to base form)
    words = [lemmatizer.lemmatize(word) for word in words]
    
    return words

# Test the preprocessing
test_text = "The paintings were absolutely AMAZING!!! I loved the colorful artworks."
processed = preprocess_for_topics(test_text)

print("Text Preprocessing Test:")
print(f"Original: {test_text}")
print(f"Processed: {processed}")
print("\nNotice: lowercase, no punctuation, lemmatized, stopwords removed")

In [ ]:
# Apply preprocessing to all museum reviews
processed_reviews = [preprocess_for_topics(review) for review in museum_reviews]

print("✅ Preprocessing complete!")
print(f"\nProcessed {len(processed_reviews)} reviews")
print("\nExample processed reviews:")
for i in range(3):
    print(f"{i+1}. {processed_reviews[i]}")

## Part 4: Building the Topic Model with Gensim

### Step 1: Create Dictionary and Corpus

Gensim needs two special data structures:
- **Dictionary**: Maps words to numeric IDs
- **Corpus**: Represents documents as bags of word IDs and frequencies

In [ ]:
# Create Gensim dictionary
dictionary = corpora.Dictionary(processed_reviews)

print("📖 Dictionary created!")
print(f"Total unique words: {len(dictionary)}")
print("\nSample word-to-ID mappings:")
for i, (word_id, word) in enumerate(list(dictionary.items())[:10]):
    print(f"  ID {word_id}: {word}")

In [ ]:
# Create corpus (bag-of-words representation)
corpus = [dictionary.doc2bow(review) for review in processed_reviews]

print("📦 Corpus created!")
print(f"Total documents: {len(corpus)}")
print("\nExample document representation (word_id, frequency):")
print(f"First review: {corpus[0]}")
print("\nHuman-readable version:")
for word_id, freq in corpus[0]:
    print(f"  '{dictionary[word_id]}' appears {freq} time(s)")

### Step 2: Train the LDA Model

**Key Parameters**:
- `num_topics`: How many topics to discover (we'll try 4)
- `passes`: How many times to go through the entire corpus (more = better but slower)
- `alpha`: Controls document-topic distribution (use 'auto' for now)
- `eta`: Controls topic-word distribution (use 'auto' for now)

In [ ]:
# Train LDA model
num_topics = 4

print(f"🤖 Training LDA model with {num_topics} topics...")
print("This may take a moment...\n")

lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_topics,
    random_state=42,  # For reproducibility
    passes=10,
    alpha='auto',
    eta='auto',
    per_word_topics=True
)

print("✅ LDA model training complete!")

## Part 5: Interpreting the Topics

### 🔍 The Critical Task: From Word Lists to Themes

The model gives you word lists with probabilities. **YOU** must interpret whether they represent coherent cultural themes!

In [ ]:
# Display the discovered topics
print("🎯 DISCOVERED TOPICS")
print("=" * 70)
print("\nEach topic shows the top 10 most important words:\n")

for idx, topic in lda_model.print_topics(num_words=10):
    print(f"Topic {idx}:")
    print(f"  {topic}")
    print(f"  Your interpretation: _____________________")
    print()

### 📝 Topic Labeling Exercise:

Look at the word lists above and assign meaningful labels:

**Topic 0**: _____________________ (What theme do these words suggest?)

**Topic 1**: _____________________ (What theme do these words suggest?)

**Topic 2**: _____________________ (What theme do these words suggest?)

**Topic 3**: _____________________ (What theme do these words suggest?)

**How did your predictions compare to the actual topics discovered?**

In [ ]:
# Create cleaner topic visualization
def display_topics_clean(model, num_words=8):
    """
    Display topics in a more readable format
    """
    for idx in range(model.num_topics):
        # Get top words for this topic
        words = model.show_topic(idx, num_words)
        
        print(f"Topic {idx}:")
        word_list = [word for word, prob in words]
        print(f"  Words: {', '.join(word_list)}")
        print()

print("🎯 TOPICS IN READABLE FORMAT")
print("=" * 40)
display_topics_clean(lda_model)

In [ ]:
# Visualize topic word weights
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx in range(num_topics):
    # Get top words and their weights
    words_weights = lda_model.show_topic(idx, 8)
    words = [word for word, weight in words_weights]
    weights = [weight for word, weight in words_weights]
    
    # Create bar chart
    axes[idx].barh(range(len(words)), weights, color='skyblue')
    axes[idx].set_yticks(range(len(words)))
    axes[idx].set_yticklabels(words)
    axes[idx].set_xlabel('Weight')
    axes[idx].set_title(f'Topic {idx}')
    axes[idx].invert_yaxis()

plt.tight_layout()
plt.show()

print("📊 Topic visualizations complete!")

## Part 6: Analyzing Document-Topic Assignments

Let's see which topics the model assigns to each document:

In [ ]:
# Get dominant topic for each document
def get_dominant_topic(ldamodel, corpus, texts):
    """
    Find the dominant topic for each document
    """
    results = []
    
    for i, doc in enumerate(corpus):
        # Get topic distribution for this document
        topic_dist = ldamodel.get_document_topics(doc)
        
        # Find dominant topic (highest probability)
        dominant_topic = max(topic_dist, key=lambda x: x[1])
        topic_num = dominant_topic[0]
        topic_prob = dominant_topic[1]
        
        # Get top words for this topic
        topic_words = [word for word, prob in ldamodel.show_topic(topic_num, 5)]
        
        results.append({
            'Document': i,
            'Dominant_Topic': topic_num,
            'Topic_Probability': round(topic_prob, 3),
            'Topic_Keywords': ', '.join(topic_words),
            'Original_Text': texts[i][:100] + '...'
        })
    
    return pd.DataFrame(results)

# Analyze documents
doc_topics = get_dominant_topic(lda_model, corpus, museum_reviews)

print("📄 DOCUMENT-TOPIC ASSIGNMENTS")
print("=" * 70)
doc_topics

In [ ]:
# Show documents grouped by topic
for topic_num in range(num_topics):
    print(f"\n📌 TOPIC {topic_num}")
    print("=" * 50)
    
    topic_docs = doc_topics[doc_topics['Dominant_Topic'] == topic_num]
    print(f"Documents in this topic: {len(topic_docs)}")
    print(f"Keywords: {topic_docs.iloc[0]['Topic_Keywords']}")
    print("\nExample texts:")
    
    for idx, row in topic_docs.head(3).iterrows():
        print(f"  - {museum_reviews[row['Document']]}")
    print()

### 💭 Critical Analysis Questions:

**Do the topic assignments make sense?**
- Look at the documents grouped under each topic
- Do they share a coherent theme?
- Where would you disagree with the model?

**What does this reveal about the algorithm's interpretation?**
- Is it detecting cultural themes or just word co-occurrence?
- What human knowledge is required to make these topics meaningful?

## Part 7: Working with Real Cultural Data

Now let's try topic modeling with a larger, more realistic cultural dataset:

In [ ]:
# Create larger sample dataset - book reviews
book_reviews_data = {
    'book': [
        'The Great Gatsby', 'The Great Gatsby', 'The Great Gatsby',
        '1984', '1984', '1984',
        'Pride and Prejudice', 'Pride and Prejudice', 'Pride and Prejudice',
        'The Hobbit', 'The Hobbit', 'The Hobbit',
        'To Kill a Mockingbird', 'To Kill a Mockingbird', 'To Kill a Mockingbird',
        'Harry Potter', 'Harry Potter', 'Harry Potter'
    ],
    'review': [
        # Gatsby reviews - themes: wealth, American Dream, symbolism
        "Fitzgerald's prose is beautiful. The symbolism of the green light and the eyes of Dr. T.J. Eckleburg represents the American Dream and moral decay of the Jazz Age.",
        "A tragic love story set in the Roaring Twenties. Gatsby's obsession with Daisy and the themes of wealth and class are timeless.",
        "The parties, the wealth, the glamour - but underneath it's about the hollowness of the American Dream. Nick's narration provides critical distance.",
        
        # 1984 reviews - themes: totalitarianism, surveillance, freedom
        "Orwell's dystopian masterpiece about totalitarian surveillance and thought control. Big Brother and the Thought Police are terrifyingly relevant today.",
        "The Party's control over language and history is chilling. Newspeak and doublethink show how authoritarian regimes manipulate truth and freedom.",
        "A warning about government surveillance and the loss of individual freedom. The torture scenes in Room 101 are unforgettable.",
        
        # Pride and Prejudice reviews - themes: marriage, class, feminism
        "Austen brilliantly satirizes class and marriage in Regency England. Elizabeth's independence and wit make her a proto-feminist heroine.",
        "The romance between Elizabeth and Darcy is wonderful, but the novel's real strength is its social commentary on women's limited options.",
        "More than a love story - it's a sharp critique of marriage as economic necessity and the restrictions on women's lives.",
        
        # Hobbit reviews - themes: adventure, fantasy, heroism
        "A perfect fantasy adventure! Bilbo's journey from comfortable hobbit to brave hero is inspiring. Dragons, dwarves, and magic rings!",
        "Tolkien's world-building is incredible. Middle-earth feels real with its own languages, cultures, and epic quests.",
        "The adventure through Mirkwood, encounters with trolls and goblins, and Smaug the dragon create an unforgettable fantasy epic.",
        
        # Mockingbird reviews - themes: racism, justice, childhood
        "Scout's childhood perspective on racial injustice in the American South is powerful. Atticus defending Tom Robinson shows moral courage.",
        "Lee confronts racism and inequality in Depression-era Alabama. The trial exposes the deep prejudice and injustice in the legal system.",
        "A coming-of-age story set against racial violence and injustice. Teaches empathy and standing up for what's right.",
        
        # Harry Potter reviews - themes: magic, friendship, good vs evil
        "The magical world of Hogwarts is enchanting! Spells, potions, and magical creatures make this fantasy unforgettable.",
        "Harry, Ron, and Hermione's friendship is the heart of the series. Their loyalty and courage in fighting Voldemort is inspiring.",
        "More than magic - it's about choosing between good and evil. The battle against dark wizards and Death Eaters is epic."
    ]
}

book_df = pd.DataFrame(book_reviews_data)

print("📚 Book Reviews Dataset Created")
print(f"Total reviews: {len(book_df)}")
print(f"Unique books: {book_df['book'].nunique()}")
print("\n🤔 Before we analyze - what topics do YOU predict will emerge?")
book_df.head()

In [ ]:
# Update stopwords for book reviews
book_stopwords = stopwords + ['book', 'novel', 'story', 'read', 'reading', 'author', 'write', 'written']

# Preprocess book reviews
def preprocess_books(text, custom_stopwords):
    text = text.lower()
    words = re.findall(r'\b[a-z]+\b', text)
    words = [word for word in words if word not in custom_stopwords and len(word) >= 3]
    words = [lemmatizer.lemmatize(word) for word in words]
    return words

processed_book_reviews = [preprocess_books(review, book_stopwords) for review in book_df['review']]

print("✅ Book reviews preprocessed!")
print(f"\nExample processed review:")
print(processed_book_reviews[0])

In [ ]:
# Create dictionary and corpus for book reviews
book_dictionary = corpora.Dictionary(processed_book_reviews)
book_corpus = [book_dictionary.doc2bow(review) for review in processed_book_reviews]

print(f"📖 Dictionary: {len(book_dictionary)} unique words")
print(f"📦 Corpus: {len(book_corpus)} documents")

In [ ]:
# Train LDA model on book reviews
num_book_topics = 5

print(f"🤖 Training LDA model with {num_book_topics} topics...\n")

book_lda = LdaModel(
    corpus=book_corpus,
    id2word=book_dictionary,
    num_topics=num_book_topics,
    random_state=42,
    passes=15,
    alpha='auto',
    eta='auto'
)

print("✅ Model training complete!\n")
print("🎯 DISCOVERED TOPICS IN BOOK REVIEWS")
print("=" * 70)

for idx in range(num_book_topics):
    words = book_lda.show_topic(idx, 10)
    word_list = [word for word, prob in words]
    print(f"\nTopic {idx}: {', '.join(word_list)}")
    print(f"Your interpretation: _____________________")

## Part 8: Experimenting with Number of Topics

### 🔬 The Critical Question: How Many Topics?

There's no "correct" number of topics! It depends on:
- Size of your dataset
- Diversity of themes
- Your research questions
- Interpretability of results

Let's try different numbers and compare:

In [ ]:
# Compare models with different topic numbers
def compare_topic_numbers(corpus, dictionary, topic_range=[3, 5, 7]):
    """
    Train models with different numbers of topics and compare
    """
    for num in topic_range:
        print(f"\n{'='*60}")
        print(f"MODEL WITH {num} TOPICS")
        print(f"{'='*60}")
        
        model = LdaModel(
            corpus=corpus,
            id2word=dictionary,
            num_topics=num,
            random_state=42,
            passes=10
        )
        
        for idx in range(num):
            words = model.show_topic(idx, 8)
            word_list = [word for word, prob in words]
            print(f"Topic {idx}: {', '.join(word_list)}")

compare_topic_numbers(book_corpus, book_dictionary, [3, 5, 7])

print("\n💭 Which number of topics gives the most interpretable results?")

## Part 9: Critical Analysis - When Topic Modeling Fails

Let's test topic modeling's limits with challenging examples:

In [ ]:
# Create challenging dataset - reviews with irony, sarcasm, mixed themes
challenging_reviews = [
    "This book was so bad it was actually entertaining. A masterpiece of terrible writing.",
    "The author clearly tried to write a thriller but accidentally created comedy gold.",
    "I loved how the romantic subplot completely contradicted the dystopian themes.",
    "The prose was beautiful but the plot made absolutely no sense whatsoever.",
    "A perfect example of how not to write a mystery novel. Thank you for this lesson."
]

print("🤔 CHALLENGING CASES FOR TOPIC MODELING")
print("=" * 50)
print("\nThese reviews contain:")
print("- Irony and sarcasm")
print("- Mixed or contradictory themes")
print("- Complex human judgment")
print("\nCan topic modeling handle these? Let's find out...\n")

# Process and analyze
processed_challenging = [preprocess_books(r, book_stopwords) for r in challenging_reviews]
challenge_dict = corpora.Dictionary(processed_challenging)
challenge_corpus = [challenge_dict.doc2bow(r) for r in processed_challenging]

challenge_lda = LdaModel(
    corpus=challenge_corpus,
    id2word=challenge_dict,
    num_topics=2,
    random_state=42,
    passes=10
)

for idx in range(2):
    words = challenge_lda.show_topic(idx, 8)
    word_list = [word for word, prob in words]
    print(f"Topic {idx}: {', '.join(word_list)}")

print("\n💭 Do these topics capture the irony and complexity?")
print("What human knowledge is required to understand these reviews?")

## Part 10: Preparing for HW4-2

You now have the skills for HW4-2! Let's review the complete workflow:

In [ ]:
# Complete workflow summary
print("🎯 HW4-2 WORKFLOW CHECKLIST")
print("=" * 50)
print("✅ 1. Review your HW4-1 predictions about topics")
print("✅ 2. Preprocess text for topic modeling (aggressive cleaning)")
print("✅ 3. Create Gensim dictionary and corpus")
print("✅ 4. Train LDA model (experiment with number of topics)")
print("✅ 5. Interpret topic word lists and assign labels")
print("✅ 6. Analyze document-topic assignments")
print("✅ 7. Compare predictions to actual discovered topics")
print("✅ 8. Integrate with HW4-1 term frequency and sentiment findings")
print("✅ 9. Reflect on complete analytical journey")

print("\n🚀 You're ready for HW4-2!")

### 💡 Key Takeaways for HW4-2:

**Technical**:
- Preprocessing for topic modeling is MORE aggressive than for sentiment
- Number of topics is a research decision, not a technical one
- Topic interpretation requires human judgment and domain knowledge

**Critical Thinking**:
- Topics are statistical patterns, not guaranteed cultural themes
- YOU must validate whether word clusters represent meaningful topics
- Some texts (irony, mixed themes) will challenge the algorithm

**Research Process**:
- Form predictions before running the model
- Test different parameter settings
- Read individual documents to validate topic assignments
- Integrate findings with previous analyses (term frequency, sentiment)

## Summary: From Words to Hidden Themes

Today you learned to:

**Technical Skills**:
- ✅ Install and use Gensim for topic modeling
- ✅ Preprocess text appropriately for LDA
- ✅ Create dictionaries and corpus representations
- ✅ Train LDA models and experiment with parameters
- ✅ Interpret topic word distributions
- ✅ Analyze document-topic assignments

**Critical Thinking**:
- ✅ Distinguish statistical patterns from cultural meanings
- ✅ Recognize limitations of algorithmic theme detection
- ✅ Question what counts as a "topic" or "theme"
- ✅ Validate computational results with close reading

**Research Skills**:
- ✅ Form and test hypotheses about hidden themes
- ✅ Integrate multiple analytical methods (frequency, sentiment, topics)
- ✅ Make interpretive decisions about model parameters
- ✅ Document analytical journey from assumptions to insights

### 🎯 Next Steps:

Apply these skills to your HW4-1 dataset in **HW4-2**, where you'll:
1. Test your topic predictions from HW4-1
2. Discover hidden themes using Gensim LDA
3. Integrate findings across all three analytical methods
4. Reflect on your complete text analysis journey

**Remember**: Being surprised by what topics emerge is a sign of genuine discovery, not analytical failure. The best insights come when data challenges our assumptions!

---

### 🔗 Critical Framework Connection: Classification Logic

Topic modeling raises fundamental questions about **how code categorizes culture**:
- Who decides what counts as a coherent "topic"?
- What cultural knowledge is embedded in stopword lists and preprocessing choices?
- How do algorithmic categories relate to human cultural understanding?
- What gets lost when we reduce texts to bags of words?

These aren't just technical questions—they're about **power, interpretation, and how computational tools shape our understanding of culture**.